# Import necessary packages

In [3]:
from rdkit import Chem
import py3Dmol
mol_file = './Molecules/25A12Rot_2.mol2'
mol = Chem.MolFromMol2File(mol_file)
mblock = Chem.MolToMolBlock(mol)
view = py3Dmol.view(width=500, height=250)
view.addModel(mblock, 'mol')
view.setStyle({'stick':{}})
view.zoomTo()
view.show()


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# Input Variables

In [4]:
n_atoms_mol = mol.GetNumAtoms()  # no. of atoms in the molecule (excluding hydrogen atoms)
print('No. of atoms: ', n_atoms_mol)
conformers = mol.GetConformers()
conf = conformers[0]
coords_mol = {}  # Coordinates of the atoms
for i in range(n_atoms_mol):
    coords_mol[i] = list(conf.GetAtomPosition(i))
print('Coordinates of atoms:')
for i in coords_mol:
    print(i, '\t:', coords_mol[i])

No. of atoms:  25
Coordinates of atoms:
0 	: [12.305, -0.778, -0.508]
1 	: [11.0, -0.01, -0.373]
2 	: [9.839, -0.938, -0.008]
3 	: [8.526, -0.154, 0.13]
4 	: [7.333, -1.002, 0.304]
5 	: [7.385, -1.885, 1.482]
6 	: [7.42, -1.187, 2.849]
7 	: [7.463, -2.211, 3.987]
8 	: [7.494, -1.531, 5.347]
9 	: [6.114, -0.178, 0.22]
10 	: [4.808, -0.978, 0.227]
11 	: [3.609, -0.075, -0.057]
12 	: [2.435, -0.883, 0.009]
13 	: [1.208, -0.308, -0.214]
14 	: [1.06, 1.058, -0.506]
15 	: [-0.199, 1.601, -0.706]
16 	: [-1.31, 0.782, -0.608]
17 	: [-1.17, -0.587, -0.317]
18 	: [-2.677, -1.439, -0.213]
19 	: [0.095, -1.206, -0.116]
20 	: [0.35, -2.78, 0.217]
21 	: [-0.849, -3.648, 0.219]
22 	: [1.181, -3.165, 1.47]
23 	: [2.134, -3.924, 0.685]
24 	: [1.449, -3.574, -0.544]


In [96]:
torsional_bonds_mol = { 0: (9, 10), 1: (19, 20)  }  # Torsional bond numbers and their respective coordinate ends
median_list_mol = [2, 6, 9, 10, 14, 19, 20, 23]  # List of atoms (median) considered for the HUBO construction
coords_rotation_mol = {11:[0], 12:[0], 13:[0], 14:[0], 15:[0], 16:[0], 17:[0], 18:[0], 19:[0], 20:[0], 21:[0,1], 22:[0,1], 23:[0,1], 24:[0,1] }  # atom index and the rotatable bond numbers list that affect them
distance_pairs_mol = {2:[14, 19, 20, 23], 6:[14, 19, 20, 23], 9:[14, 19, 20, 23], 10:[23], 14:[23], 19:[23], 20:[], 23:[] }  # indices dictionary to find distances

In [100]:
import copy

coords_median = {}
for i, index in enumerate(median_list_mol):
    coords_median[i] = coords_mol[index]
coords_median_2 = copy.deepcopy(coords_median)
print('coords_median: ', coords_median)

torsional_bonds = {}
for i in torsional_bonds_mol:
    torsional_bonds[i] = (median_list_mol.index(torsional_bonds_mol[i][0]), median_list_mol.index(torsional_bonds_mol[i][1]))
print('\ntorsional_bonds: ', torsional_bonds)

coords_rotation_dict = {}
for atom in median_list_mol:
    if atom in coords_rotation_mol.keys():
        coords_rotation_dict[median_list_mol.index(atom)] = coords_rotation_mol[atom]
print('\ncoords_rotation_dict: ', coords_rotation_dict)

distance_pairs = []
for x, y in distance_pairs_mol.items():
    u = median_list_mol.index(x)
    for i in y:
        v = median_list_mol.index(i)
        distance_pairs.append((u, v))
print('\ndistance pairs: ', distance_pairs)



coords_median:  {0: [9.839, -0.938, -0.008], 1: [7.42, -1.187, 2.849], 2: [6.114, -0.178, 0.22], 3: [4.808, -0.978, 0.227], 4: [1.06, 1.058, -0.506], 5: [0.095, -1.206, -0.116], 6: [0.35, -2.78, 0.217], 7: [2.134, -3.924, 0.685]}

torsional_bonds:  {0: (2, 3), 1: (5, 6)}

coords_rotation_dict:  {4: [0], 5: [0], 6: [0], 7: [0, 1]}

distance pairs:  [(0, 4), (0, 5), (0, 6), (0, 7), (1, 4), (1, 5), (1, 6), (1, 7), (2, 4), (2, 5), (2, 6), (2, 7), (3, 7), (4, 7), (5, 7)]


In [101]:
import sympy as sp
n_bonds = len(torsional_bonds)  # no. of bonds
n_angles = 8  # no. of discrete angles
x = sp.symbols(f'x(0:{n_bonds*n_angles})')  # hubo variables
print(x)

(x0, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15)


# Function to generate hard constraint

In [102]:
# generate hard constraint
def generate_hard_hubo():
    hard_constraint = 0
    index = 0
    for i in range(n_bonds):
        summation = 0
        for j in range(n_angles):
            summation += x[index]
            index += 1
        hard_constraint += (summation - 1) ** 2
    a_const = sp.Symbol('A_const')
    hard_constraint *= a_const
    return hard_constraint

# Functions to rotate coordinates with hubo variables

In [103]:
def rotate_coordinates(rotation_matrix, old_coords):
    coord_vector = ones(4, 1)
    coord_vector[0, 0] = old_coords[0]
    coord_vector[1, 0] = old_coords[1]
    coord_vector[2, 0] = old_coords[2]
    coord_rot_vector = sp.expand(rotation_matrix * coord_vector)
    return [coord_rot_vector[0, 0], coord_rot_vector[1, 0], coord_rot_vector[2, 0]]

In [104]:
def generate_thetas():
    angle_incr = 2 * sp.pi / n_angles
    thetas = [i*angle_incr for i in range(n_angles)]
    return thetas

In [105]:
def rotation_matrix_hubo(first_coords, second_coords, bond_no):
    x_dash, y_dash, z_dash = first_coords[0], first_coords[1], first_coords[2]
    x_ddash, y_ddash, z_ddash = second_coords[0], second_coords[1], second_coords[2]
    dx = x_ddash - x_dash
    dy = y_ddash - y_dash
    dz = z_ddash - z_dash
    l_sq = dx ** 2 + dy ** 2 + dz ** 2
    l = sp.sqrt(l_sq)
    thetas = generate_thetas()
    c_theta = 0.0
    s_theta = 0.0
    index = n_angles * bond_no
    for i in range(n_angles):
        c_theta += sp.cos(thetas[i]) * x[index]
        s_theta += sp.sin(thetas[i]) * x[index]
        index += 1
    rotation_matrix = eye(4)
    rotation_matrix[0, 0] = (dx ** 2 + (dy ** 2 + dz ** 2) * c_theta) / l_sq
    rotation_matrix[0, 1] = (dx * dy * (1 - c_theta) - dz * l * s_theta) / l_sq
    rotation_matrix[0, 2] = (dx * dz * (1 - c_theta) + dy * l * s_theta) / l_sq
    rotation_matrix[0, 3] = ((x_dash * (dy ** 2 + dz ** 2) - dx * (y_dash * dy + z_dash * dz)) * (1 - c_theta) + (
            y_dash * dz - z_dash * dy) * l * s_theta) / l_sq
    rotation_matrix[1, 0] = (dx * dy * (1 - c_theta) + dz * l * s_theta) / l_sq
    rotation_matrix[1, 1] = (dy ** 2 + (dx ** 2 + dz ** 2) * c_theta) / l_sq
    rotation_matrix[1, 2] = (dy * dz * (1 - c_theta) - dx * l * s_theta) / l_sq
    rotation_matrix[1, 3] = ((y_dash * (dx ** 2 + dz ** 2) - dy * (x_dash * dx + z_dash * dz)) * (1 - c_theta) + (
            z_dash * dx - x_dash * dz) * l * s_theta) / l_sq
    rotation_matrix[2, 0] = (dx * dz * (1 - c_theta) - dy * l * s_theta) / l_sq
    rotation_matrix[2, 1] = (dy * dz * (1 - c_theta) + dx * l * s_theta) / l_sq
    rotation_matrix[2, 2] = (dz ** 2 + (dx ** 2 + dy ** 2) * c_theta) / l_sq
    rotation_matrix[2, 3] = ((z_dash * (dx ** 2 + dy ** 2) - dz * (x_dash * dx + y_dash * dy)) * (1 - c_theta) + (
            x_dash * dy - y_dash * dx) * l * s_theta) / l_sq
    return rotation_matrix

In [107]:
from sympy.matrices import ones, eye
import time
start = time.time()

for i in coords_rotation_dict:
    rot_mat = eye(4, 4)
    for bond_no in coords_rotation_dict[i]:
        temp_rot_mat = rotation_matrix_hubo(coords_median[torsional_bonds[bond_no][0]], coords_median[torsional_bonds[bond_no][1]], bond_no)
        rot_mat = rot_mat * temp_rot_mat
    coords_median_2[i] = rotate_coordinates(rot_mat, coords_median[i])

end = time.time()
print("The time of execution: ", (end-start), "seconds")

The time of execution:  2.215198516845703 seconds


# Functions to generate the optimization contraint in hubo

In [108]:
def distance_squared(first_coords, second_coords):
    dis_sq = (second_coords[0] - first_coords[0])**2 + (second_coords[1] - first_coords[1])**2 + (second_coords[2] - first_coords[2])**2
    return dis_sq

def generate_distance_hubo():
    distance_sq = 0
    for pair in distance_pairs:
        distance_sq += distance_squared(coords_median_2[pair[0]], coords_median_2[pair[1]])
    return distance_sq.expand()

In [109]:
start = time.time()

hard_constraint = generate_hard_hubo()
distance_constraint = generate_distance_hubo()
hubo_expr = sp.expand(hard_constraint - distance_constraint)

end = time.time()
print("The time of execution: ", (end-start), "seconds")

The time of execution:  2.7050390243530273 seconds


# The full hubo expression is written in the file 'full_hubo_expr.txt'

In [110]:
def hubo_expr_to_dict():
    hubo_args = hubo_expr.args
    hubo_dict = {}
    for monom in hubo_args:
        dict_value = monom.as_coeff_mul()[0]
        monom_key = []
        monom_coeffs = monom.as_coeff_mul()[1]
        for monom_item in monom_coeffs:
            if re.match("^x(\d*)\d$", str(monom_item)):
                monom_key.append(int(str(monom_item)[1:]))
            elif re.match("^x(\d*)\\*\\*(\d)$", str(monom_item)):
                monom_key.append(int(str(monom_item).split('**')[0][1:]))
            else:
                dict_value *= monom_item
        if len(monom_key) > 0:
            monom_key.sort()
            dict_key = tuple(monom_key)
        else:
            dict_key = ()
        dict_value = dict_value.evalf()
        if dict_key in hubo_dict:
            hubo_dict[dict_key] += dict_value
        else:
            hubo_dict[dict_key] = dict_value
    return hubo_dict

In [111]:
import re
hubo_dict = hubo_expr_to_dict()

In [112]:
file_name = "hubo_dict.txt"
f = open(file_name, "w")
print('full hubo dictionary written in file - hubo_dict.txt')
f.write(str(hubo_dict))
f.close()

full hubo dictionary written in file - hubo_dict.txt


In [113]:
#This is used to check the maximum coefficient appearing in Hubo_B
A_const=0
read_dictionary_B = open(file_name, 'r').read()
HUBO_B=eval(read_dictionary_B)

#We set the Hard constraint strength as the (maximum coefficient appearing in Hubo_B)*const.
#const was empirically selected to be 10
const=10
A_const=max(map(abs, list(HUBO_B.values())))*const
# A_const = 1000

#read the final HUBO
read_dictionary= open(file_name, 'r').read()
HUBO=eval(read_dictionary)

In [114]:
print("Current size of the HUBO:",len(HUBO)) 

Current size of the HUBO: 1369


In [115]:
def threshold_approx(h, val=1):
    d =h.copy()
    monoms = h.keys()
    for m in monoms:     
        temp = d[m]
        if (temp < 0.0):
            temp = -1.0 * temp
        if (temp <= (10.0 ** (val))):
            del d[m]
    return d

In [116]:
#Coefficints with absolute value less than 10^{threshold} are deleted from the HUBO.
threshold=2

HUBO=threshold_approx(HUBO,threshold)

print("Size of the HUBO after threshold approximation:",len(HUBO))

Size of the HUBO after threshold approximation: 73


In [117]:
import dimod
#calculate the strength parameter needed by make_quadratic
max_hubo_value=max(map(abs, list(HUBO.values())))
strength=1.5*max_hubo_value
#generate the bqm
bqm = dimod.make_quadratic(HUBO, strength, dimod.BINARY)

In [118]:
def find_bond_theta_soln(solution):
    thetas = generate_thetas()
    bond_theta_soln = {}
    for key, value in solution.items():
        if solution[key] == 1:
            bond_theta_soln[key//n_angles] = thetas[key%n_angles]
    return bond_theta_soln

In [119]:
import neal
sampler = neal.SimulatedAnnealingSampler()
sample_size=10
start = time.time()
sampleset = sampler.sample(bqm, num_reads=sample_size)
end = time.time()
print("The time of execution: ", (end-start), "s")
print(sampleset)

The time of execution:  0.01391458511352539 s
   0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15      energy num_oc.
4  0  1  0  0  0  0  0  0  0  0  0  0  1  0  0  0 -936.891866       1
8  0  1  0  0  0  0  0  0  0  0  0  0  1  0  0  0 -936.891866       1
9  0  0  0  0  0  0  0  1  0  0  0  0  0  1  0  0 -866.988698       1
5  0  0  0  0  0  0  1  0  0  0  0  0  0  1  0  0  -821.04883       1
3  0  0  1  0  0  0  0  0  0  0  1  0  0  0  0  0 -795.555649       1
1  0  0  0  0  1  0  0  0  0  0  0  0  0  1  0  0 -788.330654       1
2  0  0  0  0  1  0  0  0  0  0  0  0  0  1  0  0 -788.330654       1
0  1  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  -740.03008       1
6  0  0  0  0  0  1  0  0  0  0  1  0  0  0  0  0 -717.365172       1
7  0  0  0  1  0  0  0  0  0  1  0  0  0  0  0  0 -685.299905       1
['BINARY', 10 rows, 10 samples, 16 variables]


In [120]:
sa_soln = sampleset.first.sample
print(sa_soln)
bond_theta_soln = find_bond_theta_soln(sa_soln)
print(bond_theta_soln)

{0: 0, 1: 1, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0}
{0: pi/4, 1: pi}


In [121]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler = EmbeddingComposite(DWaveSampler())
start = time.time()
sampleset = sampler.sample(bqm, num_reads=1000)
end = time.time()
print("The time of execution: ", (end-start), "s")
print(sampleset.slice(10))

The time of execution:  0.17249655723571777 s
   0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15      energy num_oc. ...
0  1  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0 -937.028813      10 ...
1  0  1  0  0  0  0  0  0  0  0  0  0  1  0  0  0 -936.891866       8 ...
2  0  0  0  0  0  0  0  1  0  0  0  0  1  0  0  0 -904.641257      14 ...
3  0  0  1  0  0  0  0  0  0  0  0  0  1  0  0  0 -904.310637       4 ...
4  1  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0 -903.981993      15 ...
5  0  1  0  0  0  0  0  0  0  0  0  1  0  0  0  0 -903.845046       9 ...
6  1  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0 -899.376254       8 ...
7  0  1  0  0  0  0  0  0  0  0  0  0  0  1  0  0 -899.239307       8 ...
8  0  0  0  0  0  0  0  1  0  0  0  1  0  0  0  0 -871.594437      33 ...
9  0  0  1  0  0  0  0  0  0  0  0  1  0  0  0  0 -871.263817       3 ...
['BINARY', 10 rows, 112 samples, 16 variables]


In [122]:
qa_soln = sampleset.first.sample
print(qa_soln)
bond_theta_soln = find_bond_theta_soln(qa_soln)
print(bond_theta_soln)

{0: 1, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0}
{0: 0, 1: pi}


In [123]:
def rotation_matrix_new_coords(first_coords, second_coords, bond_no, soln_theta):
    x_dash, y_dash, z_dash = first_coords[0], first_coords[1], first_coords[2]
    x_ddash, y_ddash, z_ddash = second_coords[0], second_coords[1], second_coords[2]
    dx = x_ddash - x_dash
    dy = y_ddash - y_dash
    dz = z_ddash - z_dash
    l_sq = dx ** 2 + dy ** 2 + dz ** 2
    l = sp.sqrt(l_sq)
    c_theta = sp.cos(soln_theta)
    s_theta = sp.sin(soln_theta)
    index = n_angles * bond_no
    rotation_matrix = eye(4)
    rotation_matrix[0, 0] = ((dx ** 2 + (dy ** 2 + dz ** 2) * c_theta) / l_sq).evalf()
    rotation_matrix[0, 1] = ((dx * dy * (1 - c_theta) - dz * l * s_theta) / l_sq).evalf()
    rotation_matrix[0, 2] = ((dx * dz * (1 - c_theta) + dy * l * s_theta) / l_sq).evalf()
    rotation_matrix[0, 3] = (((x_dash * (dy ** 2 + dz ** 2) - dx * (y_dash * dy + z_dash * dz)) * (1 - c_theta) + (
            y_dash * dz - z_dash * dy) * l * s_theta) / l_sq).evalf()
    rotation_matrix[1, 0] = ((dx * dy * (1 - c_theta) + dz * l * s_theta) / l_sq).evalf()
    rotation_matrix[1, 1] = ((dy ** 2 + (dx ** 2 + dz ** 2) * c_theta) / l_sq).evalf()
    rotation_matrix[1, 2] = ((dy * dz * (1 - c_theta) - dx * l * s_theta) / l_sq).evalf()
    rotation_matrix[1, 3] = (((y_dash * (dx ** 2 + dz ** 2) - dy * (x_dash * dx + z_dash * dz)) * (1 - c_theta) + (
            z_dash * dx - x_dash * dz) * l * s_theta) / l_sq).evalf()
    rotation_matrix[2, 0] = ((dx * dz * (1 - c_theta) - dy * l * s_theta) / l_sq).evalf()
    rotation_matrix[2, 1] = ((dy * dz * (1 - c_theta) + dx * l * s_theta) / l_sq).evalf()
    rotation_matrix[2, 2] = ((dz ** 2 + (dx ** 2 + dy ** 2) * c_theta) / l_sq).evalf()
    rotation_matrix[2, 3] = (((z_dash * (dx ** 2 + dy ** 2) - dz * (x_dash * dx + y_dash * dy)) * (1 - c_theta) + (
            x_dash * dy - y_dash * dx) * l * s_theta) / l_sq).evalf()
    return rotation_matrix

In [124]:
def compute_new_coords(solution):
    final_coords = copy.deepcopy(coords_mol)
    for i in coords_rotation_mol:
        if len(coords_rotation_mol[i]) > 0:
            rot_mat = eye(4, 4)
            for bond_no in coords_rotation_mol[i]:
                temp_rot_mat = rotation_matrix_new_coords(coords_mol[torsional_bonds_mol[bond_no][0]], coords_mol[torsional_bonds_mol[bond_no][1]], bond_no, solution[bond_no])
                rot_mat = rot_mat * temp_rot_mat
            final_coords[i] = rotate_coordinates(rot_mat, coords_mol[i])
    return final_coords


In [125]:
compute_new_coords(bond_theta_soln)

{0: [12.305, -0.778, -0.508],
 1: [11.0, -0.01, -0.373],
 2: [9.839, -0.938, -0.008],
 3: [8.526, -0.154, 0.13],
 4: [7.333, -1.002, 0.304],
 5: [7.385, -1.885, 1.482],
 6: [7.42, -1.187, 2.849],
 7: [7.463, -2.211, 3.987],
 8: [7.494, -1.531, 5.347],
 9: [6.114, -0.178, 0.22],
 10: [4.808, -0.978, 0.227],
 11: [3.60900000000000, -0.0750000000000000, -0.0570000000000000],
 12: [2.43500000000000, -0.883000000000000, 0.00900000000000000],
 13: [1.20800000000000, -0.308000000000000, -0.214000000000000],
 14: [1.06000000000000, 1.05800000000000, -0.506000000000000],
 15: [-0.199000000000000, 1.60100000000000, -0.706000000000000],
 16: [-1.31000000000000, 0.782000000000000, -0.608000000000000],
 17: [-1.17000000000000, -0.587000000000000, -0.317000000000000],
 18: [-2.67700000000000, -1.43900000000000, -0.213000000000000],
 19: [0.0950000000000000, -1.20600000000000, -0.116000000000000],
 20: [0.350000000000000, -2.78000000000000, 0.217000000000000],
 21: [1.75296098198908, -3.1709591594149

In [126]:
def find_volume_change(old_coords, new_coords):
    n = len(old_coords)
    old_distance_sq = 0
    for i in range(n-1):
        for j in range(i+1, n):
            old_distance_sq += distance_squared(old_coords[i], old_coords[j])
    
    new_distance_sq = 0
    for i in range(n-1):
        for j in range(i+1, n):
            new_distance_sq += distance_squared(new_coords[i], new_coords[j])
    print("new volume: ", new_distance_sq.evalf())
    print("old volume: ", old_distance_sq)
    print('change in volume: ', new_distance_sq.evalf() - old_distance_sq)
    return new_distance_sq.evalf() - old_distance_sq

In [127]:
bond_theta_soln = {0: 0, 1: sp.pi}
# bond_theta_soln = {0: sp.pi/4}
# bond_theta_soln = {0: 2*sp.pi/4}
# bond_theta_soln = {0: 3*sp.pi/4}
# bond_theta_soln = {0: 4*sp.pi/4}
# bond_theta_soln = {0: 5*sp.pi/4}
# bond_theta_soln = {0: 6*sp.pi/4}
# bond_theta_soln = {0: 7*sp.pi/4}

In [128]:
new_coords = compute_new_coords(bond_theta_soln)
find_volume_change(coords_mol, new_coords)

new volume:  14366.7747018963
old volume:  13633.201490000001
change in volume:  733.573211896271


733.573211896271

In [129]:
# brute force solution
start = time.time()
thetas = generate_thetas()
best_volume_change = 0
best_soln = {}
for i in range(8):
    for j in range(8):
        bond_theta_soln = {0: thetas[i], 1: thetas[j]}
        print('\nbond theta soln: ', bond_theta_soln)
        new_coords = compute_new_coords(bond_theta_soln)
        vol_difference = find_volume_change(coords_mol, new_coords)
        if vol_difference > best_volume_change:
            best_volume_change = vol_difference
            best_soln = bond_theta_soln
end = time.time()
print("The time of execution: ", (end-start), "s")    
print('best solution: ', best_soln)





bond theta soln:  {0: 0, 1: 0}
new volume:  13633.2014900000
old volume:  13633.201490000001
change in volume:  0

bond theta soln:  {0: 0, 1: pi/4}
new volume:  13733.7921249576
old volume:  13633.201490000001
change in volume:  100.590634957565

bond theta soln:  {0: 0, 1: pi/2}
new volume:  13990.3167494723
old volume:  13633.201490000001
change in volume:  357.115259472333

bond theta soln:  {0: 0, 1: 3*pi/4}
new volume:  14252.5067175862
old volume:  13633.201490000001
change in volume:  619.305227586225

bond theta soln:  {0: 0, 1: pi}
new volume:  14366.7747018963
old volume:  13633.201490000001
change in volume:  733.573211896271

bond theta soln:  {0: 0, 1: 5*pi/4}
new volume:  14266.1840669387
old volume:  13633.201490000001
change in volume:  632.982576938704

bond theta soln:  {0: 0, 1: 3*pi/2}
new volume:  14009.6594424239
old volume:  13633.201490000001
change in volume:  376.457952423933

bond theta soln:  {0: 0, 1: 7*pi/4}
new volume:  13747.4694743100
old volume:  136

In [131]:
from rdkit.Geometry import Point3D
conf = mol.GetConformer()
for i in range(len(coords_mol)):
    x, y, z = final_coords[i]
    conf.SetAtomPosition(i, Point3D(float(x), float(y), float(z)))

In [132]:
mblock = Chem.MolToMolBlock(mol)
view = py3Dmol.view(width=500, height=250)
view.addModel(mblock, 'mol')
view.setStyle({'stick':{}})
view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [133]:
mol_file = './Molecules/25A12Rot_2.mol2'
mol = Chem.MolFromMol2File(mol_file)
mblock = Chem.MolToMolBlock(mol)
view = py3Dmol.view(width=500, height=250)
view.addModel(mblock, 'mol')
view.setStyle({'stick':{}})
view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol